# NOTEBOOK DEL SQL CON EL DATASET LIMPIO

**Integrantes**

1.  *David Stivens Ordoñez*
2.   *Sebastian Montañez*
3.   *Santiago Henao Gutierrez*

## DEPENDENCIAS Y LIBRERIAS

### DEPENDENCIAS

In [1]:
%pip install PyMySQL
%pip install mysql-connector-python
%pip install PyMySQL
%pip install SQLAlchemy 
%pip install pandas pd

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached mysql_connector_python-9.0.0-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
Using cached mysql_connector_python-9.0.0-cp312-cp312-win_amd64.whl (14.3 MB)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached SQLAlchemy-2.0.35-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata

### LIBRERIAS

In [2]:
from sqlalchemy import create_engine # type: ignore
from sqlalchemy.exc import SQLAlchemyError # Nos servirá para validar la conexión # type: ignore
import pymysql # type: ignore
import pandas as pd # type: ignore
import mysql.connector # type: ignore
import mysql.connector # Importa el módulo para conectar con MySQL # type: ignore
from mysql.connector import errorcode # Importa los códigos de error de MySQL # type: ignore
import pandas as pd # Importa pandas para manejar datos # type: ignore
import mysql.connector # Importa el conector de MySQL # type: ignore
from mysql.connector import errorcode # type: ignore # Importa los códigos de error de MySQL

## BASE DE DATOS

### DATOS DE CREACION BD

In [3]:
user = "root"
password = "708090"
host = 'localhost' #ESTO SE CAMBIA SI ES REMOTO
database_name = 'db_superstore'

In [4]:
config = {
    'user': user,
    'password': password,
    'host': host,
    'database': database_name,
    'raise_on_warnings': True
}

### SQL (CREACION DATABASE Y TABLAS)

In [5]:
try:
    # Conectar al servidor MySQL sin especificar base de datos
    conn = mysql.connector.connect(
        host=config['host'],
        user=config['user'],
        password=config['password']
        # No se especifica la base de datos aquí
    )
    cursor = conn.cursor()

    # Verificar si la base de datos ya existe
    cursor.execute(f"SHOW DATABASES LIKE '{database_name}';")
    database_exists = cursor.fetchone()

    if not database_exists:
        # Crear la base de datos si no existe
        cursor.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos '{database_name}' creada exitosamente.")
    else:
        print(f"La base de datos '{database_name}' ya existe.")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrectos.")
    elif err.errno == errorcode.ER_NO_SUCH_TABLE:
        print("La tabla no existe.")
    else:
        print(err)

finally:
    # Verificar si 'conn' está definido y si está conectado antes de cerrar
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("Conexión cerrada.")

La base de datos 'db_superstore' ya existe.
Conexión cerrada.


In [6]:
try:
    # Conectar al servidor MySQL usando la configuración especificada
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # Seleccionar la base de datos
    cursor.execute(f"USE {database_name};")

    # Crear las tablas si no existen
    create_table_Orders_query = """
    CREATE TABLE IF NOT EXISTS Orders (
        Order_ID VARCHAR(50),
        Order_Date DATE,
        Ship_Date DATE,
        Ship_Mode VARCHAR(50),
        Customer_ID VARCHAR(50),
        Customer_Name VARCHAR(200),
        Segment VARCHAR(50),
        City VARCHAR(50),
        State VARCHAR(50),
        Country VARCHAR(50),
        Region VARCHAR(50),
        Market VARCHAR(50),
        Product_ID VARCHAR(20),
        Category VARCHAR(50),
        Sub_Category VARCHAR(50),
        Product_Name VARCHAR(255),
        Sales FLOAT,
        Quantity INT,
        Discount FLOAT,
        Profit FLOAT,
        Shipping_Cost FLOAT,
        Order_Priority VARCHAR(50)
    );
    """

    create_table_People_query = """
    CREATE TABLE IF NOT EXISTS People (
        Person VARCHAR(50),
        Region VARCHAR(50)
    );
    """

    create_table_Returns_query = """
    CREATE TABLE IF NOT EXISTS Returns (
        Order_ID VARCHAR(50),
        Region VARCHAR(50)
    );
    """

    cursor.execute(create_table_Orders_query)
    print("Tabla 'Orders' creada exitosamente o ya existe.")

    cursor.execute(create_table_Returns_query)
    print("Tabla 'Returns' creada exitosamente o ya existe.")

    cursor.execute(create_table_People_query)
    print("Tabla 'People' creada exitosamente o ya existe.")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrectos.")
    elif err.errno == errorcode.ER_NO_SUCH_TABLE:
        print("La tabla no existe.")
    else:
        print(err)

finally:
    conn.close()

1050: Table 'orders' already exists


### CARGUE DE DATOS

In [7]:
df_Orders_Cleaned = "C:/Users/dqvdd/OneDrive - Universidad Autonoma de Occidente/UAO 2024-03 - BIG DATA/GESTION Y ALMACENAMIENTO DE DATOS/PROYECTO/df_Orders_Cleaned.csv"
df_Returns_Cleaned = "C:/Users/dqvdd/OneDrive - Universidad Autonoma de Occidente/UAO 2024-03 - BIG DATA/GESTION Y ALMACENAMIENTO DE DATOS/PROYECTO/df_Returns_Cleaned.csv"
df_People_Cleaned = "C:/Users/dqvdd/OneDrive - Universidad Autonoma de Occidente/UAO 2024-03 - BIG DATA/GESTION Y ALMACENAMIENTO DE DATOS/PROYECTO/df_People_Cleaned.csv"

In [8]:
try:
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database_name}"
    engine = create_engine(connection_string)

    df_orders = pd.read_csv(df_Orders_Cleaned, sep=';')
    df_orders_to_sql = df_orders.to_sql('orders', engine, if_exists='replace', index=False)

    df_returns = pd.read_csv(df_Returns_Cleaned, sep=';')
    df_returns_to_sql = df_returns.to_sql('returns', engine, if_exists='replace', index=False)

    df_people = pd.read_csv(df_People_Cleaned, sep=';')
    df_people_to_sql = df_people.to_sql('people', engine, if_exists='replace', index=False)

except mysql.connector.Error as err:
    print(f"Error al cargar datos en la tabla 'people': {err}")
finally:
    print("datos cargados")

datos cargados
